In [484]:
# CHECK CATEGOIRES FUNCTION
def checkCategorical(data_frame, column, approved_list):
    _counts = df[column].value_counts()
    not_in_list = 0
    for x in _counts.index.tolist():
        if x not in approved_list: 
            print(x)
            not_in_list += 1
    print('{} {} categories found that were not in the official {} list.'.format(not_in_list, column, column))
    print(_counts)

In [485]:
# PLOT CONTINUOUS DATA FUNCTION
def plotContinuous(data_frame, x, y, marker_size, size):
    groups = data_frame.groupby('y')
    p = 0
    for name, group in groups:
        plt.plot(group[x],group[y],marker='.',ms=marker_size[p],linestyle='',label=name)
        p += 1
    plt.rcParams["figure.figsize"] = (size[0],size[1])
    plt.xlabel=x
    plt.ylabel=y
    plt.legend(prop={'size':20})
    plt.show()

In [486]:
# MLP WRAPPER
def mlpWrapper(df, mute):
    # split into train and test data
    if not mute: print('TTT seed value: {}'.format(TTT_seed))
    train, test = train_test_split(
        df,
        test_size=0.2,
        random_state=TTT_seed
    )
    X_train = train[train.columns[:-1]]
    X_test = test[test.columns[:-1]]
    y_train = train[train.columns[-1]]
    y_test = test[test.columns[-1]]

    # useful info print out
    if not mute: print('training set size: {}'.format(train.shape[0]))
    if not mute: print('testing set size: {}'.format(test.shape[0]))

    # call training function
    if not mute: print('MLP seed value: {}'.format(MLP_seed))
    mlp = MLPClassifier(
        hidden_layer_sizes=50,
        activation='logistic',
        learning_rate_init=0.4,
        batch_size=100,
        solver='sgd',
        random_state=MLP_seed,
        max_iter=10000
    ).fit(X_train,y_train)

    # print out results
    result = mlp.predict(test[test.columns[:-1]])
    result_train = mlp.score(X_train, y_train)
    result_test = mlp.score(X_test, y_test)
    if not mute: print('Training accuracy: {:.2f}'.format(result_train))
    if not mute: print('Testing accuracy: {:.2f}'.format(result_test))

    # return
    return {'train': result_train, 'test': result_test}



In [487]:
# 1 HOT
def oneHot(df, column):

    # 1 hot encoding
    _df = pd.get_dummies(df, columns=[column], prefix=column)
    
    #return
    return _df

In [488]:
# INSERT
def insert(df, insert_df):
    _left = df[df.columns[:-1]]
    _right = df[df.columns[-1]]
    _final = pd.concat([_left,insert_df,_right],axis=1)
    return _final

In [489]:
# REMOVE FIRST TIME CALLS
def removeFirstCalls(df):
    _df = df
    return _df[_df['pdays']!=999]

In [490]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from random import randint
import random
import pprint

In [491]:
# LOAD DATA
file_name = '../bank-additional/bank-additional/bank-additional-full.csv'
df = pd.read_csv(file_name, sep=';')
df = df.drop_duplicates(keep='first')
TTT_seed = randint(0,99)
MLP_seed = randint(0,99)
_ready = {}
pp = pprint.PrettyPrinter(indent=4)

In [492]:
# REMOVE PDAYS

def removePdays(df):
    return df.drop('pdays', axis=1)

In [493]:
# MLP NORMALIZED DATAFRAME 

def getNormalized(df, remove_pdays):

    # selection list for readability
    selection = [
        'age',
        'campaign',
        'pdays',
        'previous',
        'emp.var.rate',
        'cons.price.idx',
        'cons.conf.idx',
        'euribor3m',
        'nr.employed',
        'y'
    ]
    if remove_pdays: selection.remove('pdays')

    # scrub dataframe: no first selection and no first time subs
    _df = df[selection]

    # max/min normalization
    _norm = _df[_df.columns[:-1]]
    _norm = (_norm - _norm.mean()) / (_norm.max() - _norm.min())
    
    # reassemble
    _final = pd.concat([_norm, _df['y']], axis=1)
    
    #return
    return _final

In [494]:
# # MLP ONLY FIRST CALL SUBS

# # selection list for readability
# selection = [
#     'age',
#     'duration',
#     'campaign',
#     'pdays',
#     'previous',
#     'emp.var.rate',
#     'cons.price.idx',
#     'cons.conf.idx',
#     'euribor3m',
#     'nr.employed',
#     'y'
# ]

# # scrub dataframe: no first selection and only first call subs
# scrubbed = df[selection]
# scrubbed = scrubbed[scrubbed['pdays']==999]

# # max/min normalization
# norm_df = scrubbed[selection[:-1]]
# norm_df = norm_df.loc[:, norm_df.columns != 'pdays']
# norm_df = (norm_df-norm_df.mean())/(norm_df.max()-norm_df.min())
# final = pd.concat([norm_df,scrubbed['pdays'],scrubbed['y']],axis=1)

# # split into train and test data
# train,test = train_test_split(final,test_size=0.999, random_state=24)
# X_train = train[train.columns[:-1]]
# X_test = test[test.columns[:-1]]
# y_train = train[train.columns[-1]]
# y_test = test[test.columns[-1]]

# # useful info print out
# print('training set size: {}'.format(train.shape[0]))
# print('testing set size: {}'.format(test.shape[0]))

# # call training function
# mlp = MLPClassifier(
#     hidden_layer_sizes=1,
#     activation='logistic',
#     learning_rate_init=0.04,
#     batch_size=10,
#     solver='sgd',
#     random_state=0,
#     max_iter=10000
# ).fit(X_train,y_train)

# # print out results
# result = mlp.predict(test[test.columns[:-1]])
# print('Training accuracy: {:.2f}'.format(mlp.score(X_train, y_train)))
# print('Testing accuracy: {:.2f}'.format(mlp.score(X_test, y_test)))

In [495]:
# THE PROBLEM WITH FIRST TIME CALL DATA
counts = df.pdays.value_counts()
print('First time calls represented {:.0%} of the total number of calls.'.format(counts.max()/df.shape[0]))
no = df[df['y']=='no']
counts = no.pdays.value_counts()
print('{:.0%} of the calls were first time calls where the customer declined the offer.'.format(counts.max()/df.shape[0]))

First time calls represented 96% of the total number of calls.
87% of the calls were first time calls where the customer declined the offer.


In [496]:
# MLP ENCODED JOB

def getJob(df):

    # 1 hot encoding
    _jobs = df['job']
    
    # print(_jobs.value_counts())
    _jobs = oneHot(_jobs, 'job')

    # output
    # print(_jobs.head())

    return _jobs

In [497]:
# MLP ENCODED MARITAL

def getMarital(df):

    # 1 hot encoding
    _marital = df['marital']
    # print(_marital.value_counts())
    _marital = oneHot(_marital, 'marital')
    _marital = _marital.drop('marital_unknown',axis=1)

    # output
    # print(_marital.head())

    return _marital

In [498]:
# MLP ENCODED EDUCATION

def getEducation(df):

    # replace
    _encoded_education = {
        'education': {
            'unknown': 0/7,
            'illiterate': 1/7,
            'basic.4y': 2/7,
            'basic.6y': 3/7,
            'basic.9y': 4/7,
            'high.school': 5/7,
            'professional.course': 6/7,
            'university.degree': 7/7,
        }
    }
    # print(_education.education.value_counts())
    _education = df.replace(_encoded_education)
    _education = _education.education

    # output
    # print(_education.head())

    return _education

In [499]:
# MLP ENCODED DEFAULT

def getDefault(df):

    # 1 hot encoding
    _default = df.default
    # print(_default.value_counts())
    _default = oneHot(_default, 'default')
    _default = _default.drop('default_unknown',axis=1)

    # output
    # print(_default.head())

    return _default

In [500]:
# MLP ENCODED HOUSING

def getHousing(df):

    # 1 hot encoding
    _housing = df.housing
    # print(_housing.value_counts())
    _housing = oneHot(_housing, 'housing')
    _housing = _housing.drop('housing_unknown',axis=1)

    # output
    # print(_housing.head())

    return _housing

In [501]:
# MLP ENCODED LOAN

def getLoan(df):

    # 1 hot encoding
    _loan = df.loan
    # print(_loan.value_counts())
    _loan = oneHot(_loan, 'loan')
    _loan = _loan.drop('loan_unknown',axis=1)

    # output
    # print(_loan.head())

    # return
    return _loan

In [502]:
# MLP ENCODED CONTACT

def getContact(df):

    # 1 hot encoding
    _contact = df.contact
    # print(_contact.value_counts())
    _contact = oneHot(_contact, 'contact')

    # output
    # print(_contact.head())

    # return
    return _contact

In [503]:
# MLP ENCODED MONTH

def getMonth(df):

    # 1 hot encoding
    _month = df.month
    # print(_month.value_counts())
    _month = oneHot(_month, 'month')

    # output
    # print(_month.head())

    # return
    return _month

In [504]:
# MLP ENCODED DAY OF WEEK

def getDayOfWeek(df):

    # 1 hot encoding
    _day_of_week = df.day_of_week
    # print(_day_of_week.value_counts())
    _day_of_week = oneHot(_day_of_week, 'day_of_week')

    # output
    # print(_day_of_week.head())

    # return
    return _day_of_week

In [505]:
# MLP ENCODED PREVIOUS OUTCOME

def getPoutcome(df):

    # 1 hot encoding
    _poutcome = df.poutcome
    # print(_poutcome.value_counts())
    _poutcome = oneHot(_poutcome, 'poutcome')

    # output
    # print(_poutcome.head())

    # return
    return _poutcome

In [506]:
# ADD ENCODED DATA
def assembleData(normalized, ready):
    all_data = normalized
    for key in ready:
        if key != 'normalized':
            # print('adding {}...'.format(key))
            all_data = insert(all_data, ready[key])
    return all_data

In [507]:
# ASEMBLE DATA

def assembleReady(df, remove_first_call, remove_pdays):
    ready = {}
    if remove_first_call and not remove_pdays: _df = removeFirstCalls(df)
    if not remove_first_call and remove_pdays: _df = removePdays(df)
    if not remove_first_call and not remove_pdays: _df = df
    ready['normalized'] = getNormalized(_df, remove_pdays)
    ready['job'] = getJob(_df)
    ready['marital'] = getMarital(_df)
    ready['education'] = getEducation(_df) 
    ready['default'] = getDefault(_df)
    ready['housing'] = getHousing(_df)
    ready['loan'] = getLoan(_df)
    ready['contact'] = getContact(_df)
    ready['month'] = getMonth(_df)
    ready['day_of_week'] = getDayOfWeek(_df)
    ready['poutcome'] = getPoutcome(_df)
    return ready

In [508]:
# RUN 100 TIMES AND SAVE RESULTS
_all_runs = []
_ready = assembleReady(df, True, False)
_all_data = assembleData(_ready['normalized'], _ready)
for _i in range(100):
    if _i % 10 == 0: print('running test {}'.format(_i+1))
    _this_run = {}
    random.seed()
    TTT_seed = randint(0,99)
    MLP_seed = randint(0,99)
    _this_run['TTT_seed'] = TTT_seed
    _this_run['MLP_seed'] = MLP_seed
    _results = mlpWrapper(_ready['normalized'], True)
    _this_run['normalized_train'] = _results['train']
    _this_run['normalized_test'] = _results['test']
    _results = mlpWrapper(_all_data, True)
    _this_run['all_train'] = _results['train']
    _this_run['all_test'] = _results['test']
    _all_runs.append(_this_run)
all_runs_df = pd.DataFrame(_all_runs)
# TEST RESULTS
print(all_runs_df.normalized_test.mean())
print(all_runs_df.all_test.mean())

(1515, 12)
(1515, 3)
(1515,)
(1515, 1)
(1515, 2)
(1515, 2)
(1515, 2)
(1515, 10)
(1515, 5)
(1515, 2)
running test 1
0.6006600660066006
0.5858085808580858


In [510]:
# RUN 100 TIMES AND SAVE RESULTS
_all_runs = []
_ready = assembleReady(df, False, True)
all_data = assembleData(_ready['normalized'], _ready)
for _i in range(2):
    if _i % 10 == 0: print('running test {}'.format(_i+1))
    _this_run = {}
    random.seed()
    TTT_seed = randint(0,99)
    MLP_seed = randint(0,99)
    _this_run['TTT_seed'] = TTT_seed
    _this_run['MLP_seed'] = MLP_seed
    _results = mlpWrapper(_ready['normalized'], True)
    _this_run['normalized_train'] = _results['train']
    _this_run['normalized_test'] = _results['test']
    _results = mlpWrapper(_all_data, True)
    _this_run['all_train'] = _results['train']
    _this_run['all_test'] = _results['test']
    _all_runs.append(_this_run)
all_runs_df = pd.DataFrame(_all_runs)
# TEST RESULTS
print(all_runs_df.normalized_test.mean())
print(all_runs_df.all_test.mean())

(41176, 21)
(41176, 12)
(41176, 3)
(41176,)
(41176, 2)
(41176, 2)
(41176, 2)
(41176, 2)
(41176, 10)
(41176, 5)
(41176, 3)
running test 1
0.8898130160271978
0.6287128712871287
